<a href="https://www.kaggle.com/code/moajjem04/titanic-streamlit?scriptVersionId=100576511" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Modules

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, MissingIndicator
# Feature Engineering
from sklearn.decomposition import PCA
# Model Building
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# Inference
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score,f1_score
# Save
import pickle

# List All Files

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Read Files

In [3]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data Preprocessing

In [4]:
y = train_df['Survived']

In [5]:
cols_to_use = ['Pclass','Sex','Age','SibSp','Parch']
X = train_df[cols_to_use]
X.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,male,22.0,1,0
1,1,female,38.0,1,0
2,3,female,26.0,0,0
3,1,female,35.0,1,0
4,3,male,35.0,0,0


## Check Missing Values

In [6]:
X.isna().sum()

Pclass      0
Sex         0
Age       177
SibSp       0
Parch       0
dtype: int64

In [7]:
age_median = X['Age'].median()
age_median

28.0

# Fill Missing values

In [8]:
X['Age'] = X['Age'].fillna(age_median)
X.isna().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
dtype: int64

In [9]:
X.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,male,22.0,1,0
1,1,female,38.0,1,0
2,3,female,26.0,0,0
3,1,female,35.0,1,0
4,3,male,35.0,0,0


# Adding Extra Feature

In [10]:
temp = X.copy()
temp['Is_alone'] = 0
temp.loc[X['SibSp'] > 0, 'Is_alone'] = 1
temp.loc[X['Parch'] > 0, 'Is_alone'] = 1
temp.head()

,Pclass,Sex,Age,SibSp,Parch,Is_alone
0,3,male,22.0,1,0,1
1,1,female,38.0,1,0,1
2,3,female,26.0,0,0,0
3,1,female,35.0,1,0,1
4,3,male,35.0,0,0,0


In [11]:
X = temp.copy()
X.head()

,Pclass,Sex,Age,SibSp,Parch,Is_alone
0,3,male,22.0,1,0,1
1,1,female,38.0,1,0,1
2,3,female,26.0,0,0,0
3,1,female,35.0,1,0,1
4,3,male,35.0,0,0,0


Drop `SibSp` and `Parch`

In [12]:
X = X.drop(['SibSp','Parch'],axis = 1)
X.head()

,Pclass,Sex,Age,Is_alone
0,3,male,22.0,1
1,1,female,38.0,1
2,3,female,26.0,0
3,1,female,35.0,1
4,3,male,35.0,0


# Scale Data

In [13]:
num_feat = ['Age']
cat_feat = ['Pclass','Sex','Is_alone']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore',sparse = False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_feat),
        ('cat', categorical_transformer, cat_feat)
    ]
)

In [14]:
X_transformed = preprocessor.fit_transform(X)
X_transformed.shape

(891, 8)

# Train

In [15]:
dtree_pipe = Pipeline([('prep',preprocessor),('tree',DecisionTreeClassifier())])
tree = DecisionTreeClassifier()

In [16]:
res = cross_val_score(estimator = tree,X = X_transformed,y = y,scoring = 'f1')
np.mean(res),np.std(res)

(0.7223076079100099, 0.020045979857644548)

In [17]:
tree.fit(X_transformed,y)

DecisionTreeClassifier()

# Save Model

In [18]:
model_dict = {"tree_model": tree, "preprocess" : preprocessor}
pickle.dump(model_dict,open('tree.pkl','wb'))

# Submission

In [19]:
sample = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
sample.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [20]:
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [21]:
X_test = test_df[cols_to_use]
#temp = X.copy()
X_test['Is_alone'] = 0
X_test.loc[X_test['SibSp'] > 0, 'Is_alone'] = 1
X_test.loc[X_test['Parch'] > 0, 'Is_alone'] = 1
X_test = X_test.drop(['SibSp','Parch'],axis = 1)
X_test['Age'] = X_test['Age'].fillna(age_median)
print(X_test.isna().sum())
X_test.head()

Pclass      0
Sex         0
Age         0
Is_alone    0
dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,Pclass,Sex,Age,Is_alone
0,3,male,34.5,0
1,3,female,47.0,1
2,2,male,62.0,0
3,3,male,27.0,0
4,3,female,22.0,1


In [22]:
X_test_proc = preprocessor.transform(X_test)
X_test_proc.shape

(418, 8)

In [23]:
y_pred = tree.predict(X_test_proc)
submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':y_pred})
submission.to_csv('tree_submission.csv', index=False)